In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from pyspark.sql import SparkSession

In [ ]:
pd.set_option('display.max_columns', int(1e7))
pd.set_option('display.max_rows', int(1e7))
pd.set_option('display.width', int(1e7))

------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Defining OpheliaSpark Session for pseudo-distributed computing:

In [ ]:
spark = SparkSession.builder.appName('Risk_Parity').getOrCreate()
sc = spark.sparkContext
sc

# Reading persisted Portfolio Yields dataframe:

In [ ]:
portfolio_yield_window_path = '/data/core/fince/data/portfolioOptimization/portfolio_yield_window/'
portfolio_yield_df = spark.read.parquet(portfolio_yield_window_path)

In [ ]:
portfolio_yield_df.limit(5).toPandas()

In [ ]:
len(portfolio_yield_df.columns)

# From now on we construct yield portfolio matrix for Risk Parity process.

### Get model parameters Class:
#### This calss will have defined functions that will help us as utilities for creating specific matrices objects.

In [ ]:
class GeneratorMat():
    
    def __init__(self, df, cols=10):
        """
        Constructor object for input parameters.
        :param df: OpheliaSpark DataFrameUtils input with asset price
        :param cols: Number of columns to optimize, 10 columns default.        
        """
        dismiss_cols = len(df.columns) - cols
        field_array = df.columns[:-dismiss_cols]
        
        monthly_return = np.array(df.select(*field_array).collect())
        
        self.numpy_matrix = monthly_return

In [ ]:
numpy_matrix = GeneratorMat(df=portfolio_yield_df).numpy_matrix
numpy_matrix

### Then, lets fill with zeros *(initialize)* **covariance matrix** with dimensions $(M x M)$, and **weight Risk Parity** matrix with $(T_2 x M)$.
#### *$M$ = as Total columns from portfolio funds data (211).*
#### *$T$ = as Tistorical yeilds from portfolio funds data (931).*

In [ ]:
class ModelParameters():
    
    def rowsMat():
        """
        Gets numpy.array matrix shape.
        :return: number matrix shape dim (rows).
        """
        if isinstance(numpy_matrix, (np.ndarray)):
            n_rows = numpy_matrix.shape[0]
        else:
            print("matrix not valid, must be numpy.array object with float terms.")
        
        return n_rows
    
    def columnsMat():
        """
        Gets numpy.array matrix shape.
        :return: number matrix shape dim (cols).
        """
        if isinstance(numpy_matrix, (np.ndarray)):
            cols = numpy_matrix.shape[1]
        else:
            print("matrix not valid, must be numpy.array object with float terms.")
        
        return cols
    
    def timePrediction(numpy_matrix, time_predict=None, n_month=None):
        """
        Set time prediction function it creates an returns time prediction window, 
        starting month variable for analysis and historical observations variable.
        :param numpy_matrix: yield portfolio numpy.array matrix.
        :param time_predict: time to predict int, 11 historical months default.
        :param n_month: number of months int, 1 month to predict default.
        :return: tuple with initial model optimization params.
        """
        n_rows = ModelParameters.rowsMat()
            
        if time_predict is None:
            time_predict = 11
        if n_month is None:
            n_month = 1

        if time_predict is not None:
            if isinstance(time_predict, int):
                time_predict = time_predict
            else:
                print("time predict params not valid, must be integer.")
        
        if n_month is not None:
            if isinstance(n_month, int):
                n_month =  n_month
            else:
                print("number of months params not valid, must be integer.")
            
        start_month = time_predict + n_month
        time_observed = n_rows - start_month
        end_month = n_rows
          
        return start_month, end_month, time_predict, time_observed
    
    def zeroCovMat(cols):
        """
        Create zero matrix initializes a zero filled numpy.array matrix with adapting dimensions.
        :param n_cols: int number columns N (fund target).
        :param time_observed: time to analyze, int.
        :return: tuple with initial zero filled numpy.array objects.
        """
        if isinstance(cols, int):
            cols = cols
        else:
            print("number of columns params not valid, must be integer.")
        
        return np.zeros((cols, cols))
    
    def zeroWeightMat(time_observed, cols):
        """
        Create zero matrix initializes a zero filled numpy.array matrix with adapting dimensions.
        :param n_cols: int number columns N (fund target).
        :param time_observed: time to analyze, int.
        :return: tuple with initial zero filled numpy.array objects.
        """
        if isinstance(cols, int):
            cols = cols
        else:
            print("number of columns params not valid, must be integer.")
            return None
        
        if isinstance(time_observed, int):
            time_observed = time_observed
        else:
            print("time observed params not valid, must be integer.")
            return None
        
        return np.zeros((time_observed, cols))
    
    def matT(numpy_matrix):
        """
        Matrix transpose creates an numpy.array object to its transposed shape.
        :param numpy_matrix: yield portfolio numpy.array matrix.
        :return: numpy.array object transposed.
        """
        if not isinstance(numpy_matrix, (np.ndarray)):
            print("matrix not valid, must be numpy.array object with float terms.")
            return None
            
        return numpy_matrix.T
    
    def oneNMat(cols):
        """
        One/n creates a equeally weighted numpy.array matrix.
        :param n_cols: int number columns N (fund target).
        :return: numpy.array object Onen/n weights with (1 x N) dimensions.
        """
        if isinstance(cols, int):
            cols = cols
        else:
            print("number of columns params not valid, must be integer.")
            return None
        
        return np.full((1, cols), 1/cols)
    
    def zeroFillMat(list_vector, time_observed):
        """
        Zero Filled Vector creates a zero filled numpy.array 1-column matrix, aka: zero-vector.
        :param list_vector: array type with string names for different type of vector.
        :param time_observed: int time window observations.
        :return: numpy.array objectZero Filled Vector with (N x 1) dimensions.
        """
        if isinstance(time_observed, int):
            time_observed = time_observed
        else:
            print("time observed params not valid, must be integer.")
            return None
        
        mapped_vec = {}
        for i in list_vector:
            mapped_vec[i] = np.zeros((time_observed, 1))
        
        return mapped_vec

### Now lets define $M$ and $N$ parameters for optimization model, that will be the same as total columns and total rows; respectively.
#### *Start month window: as the start of historical analysis for prediction.*
#### *End month window: as the end of historical analysis for prediction.*
#### *$T_1$: as the Timing 1 for prediction window (11 months).*
#### *$T_2$: as the Timing 2 for window observations (931 unique dates).*

**Note 1: Keep on-track with *size* variable, such that is the total count of historical portfolio data (943), and *$T_2$* as the total time to analyze (931), the mathematical difference between this two will be the time to predict (12 months).** 

In [ ]:
N = ModelParameters.rowsMat() # N rows of matrix
M = ModelParameters.columnsMat() # M columns of matrix
start_month, end_month, T1, T2 = ModelParameters.timePrediction(numpy_matrix=numpy_matrix)
covariance_matrix = ModelParameters.zeroCovMat(cols=M)

class RiskParityComputing(object):
    
    def covaMat():
        """
        covaMat creates a covariance matrix numpy.array, no arguments needed.
        :return: numpy.array Covariance matrix with (M x M) dimensions.
        """
        for y in range(start_month, end_month):
            covariance_matrix = np.cov(ModelParameters.matT(numpy_matrix)[:,y - T1:y])
        return covariance_matrix
    
    def rewMat():
        """
        rewMat creates an equal weight matrix numpy.array for computing returns, no arguments needed.
        :return: numpy.array equal weight matrix with (N x M) dimensions.
        """
        return_equal_weight  = ModelParameters.zeroWeightMat(time_observed=T2, cols=M)
        for y in range(start_month, end_month):
            return_equal_weight[y - start_month] = np.dot(numpy_matrix[y, :], 1 / N)
        return return_equal_weight
    
    def retEWMat():
        """
        retEWMat creates the sum of matrix numpy.array multiplications, no arguments needed.
        :return: numpy.array return Equally Weight matrix with (N x 1) dimensions.
        """
        return_Equally_Weight = ModelParameters.zeroFillMat(list_vector=["retEW"], time_observed=T2)["retEW"]
        for y in range(start_month, end_month):
            return_Equally_Weight[y - start_month] = sum(RiskParityComputing.rewMat()[y - start_month])
        return return_Equally_Weight
    
    def varianceMat(x):
        """
        varianceMat helps to compute the variance produced x.T cov[X_i, X_j] x.
        :param x: X_ij elements from input numpy.array with dimensions (M,).
        :return: numpy.float64 variance.
        """
        return (x.T @ covariance_matrix @ x)
    
    def sigmaMat(x):
        """
        sigmaMat computes sigma produced by varianceMat^0.5.
        :param x: X_ij elements from input numpy.array with dimensions (M,).
        :return: numpy.float64 sigma.
        """
        return (RiskParityComputing.varianceMat(x) ** .5)
    
    def marginalRiskContribution(x):
        """
        marginalRiskContribution computes from sigmaMat method.
        :param x: X_ij elements from input numpy.array with dimensions (M,).
        :return: numpy.array Covariance matrix with (M,) dimensions.
        """
        return (1 / RiskParityComputing.sigmaMat(x) * (covariance_matrix @ x))
    
    def marginalRiskProduct(x):
        """
        marginalRiskProduct creates a wieghts of Risk Contribution matrix numpy.array.
        :param x: X_ij elements from input numpy.array with dimensions (M,).
        :return: numpy.array weights of proportion of contribution with (M,) dimensions.
        """
        return (x * RiskParityComputing.marginalRiskContribution(x))
    
    def riskContribution(x):
        """
        riskContribution creates a weights of Risk Contribution matrix numpy.array.
        :param x: X_ij elements from input numpy.array with dimensions (M,).
        :return: numpy.array weights of proportion of contribution with (M,) dimensions.
        """
        risk_contribution = RiskParityComputing.marginalRiskProduct(x)
        return (risk_contribution / risk_contribution.sum())
    
    def riskParityObjective(x):
        """
        riskParityObjective computes risk parity objective.
        :param x: X_ij elements from input numpy.array with dimensions (M,).
        :return: numpy.float64 risk parity objective.
        """
        reshape_matrix = np.reshape(RiskParityComputing.marginalRiskProduct(x), (len(x), 1))
        risk_diffs = reshape_matrix - reshape_matrix.T
        return np.sum(np.square(np.ravel(risk_diffs)))
    
    def maxDivObjective(x):
        """
        maxDivObjective computes maximum division parity objective.
        :param x: X_ij elements from input numpy.array with dimensions (M,).
        :return: numpy.float64 maximum division parity objective negative number.
        """
        weights_volume = np.dot(np.sqrt(np.diag(covariance_matrix)), x.T)
        diver_ratio = weights_volume / RiskParityComputing.sigmaMat(x)
        return -diver_ratio
    
    def weightSumConstraint(x):
        """
        weightSumConstraint computes sum of all elments called 'constraints'.
        :param x: X_ij elements from input numpy.array with dimensions (M,).
        :return: numpy.float64 total sum of all constraints.
        """
        return (x.sum() - 1.0)
        
    def weightLongonly(x):
        """
        weightLongonly pass the risk contribution matrix numpy.array.
        :param x: X_ij elements from input numpy.array with dimensions (M,).
        :return: numpy.array weightLongonly matrix with (M,) dimensions.
        """
        return (x)

# Generator function for Risk Contribution variables
- mrc aka: marginal risk contribution
- rc aka: risk contribution

In [ ]:
class PortfolioOptimization(object):
    
    def runRiskParity():
        covariance_matrix = RiskParityComputing.covaMat()
        x0 = np.repeat(1 / covariance_matrix.shape[1], covariance_matrix.shape[1])
        constraints = (
            {'type': 'eq', 'fun': RiskParityComputing.weightSumConstraint},
            {'type': 'ineq', 'fun' : RiskParityComputing.weightLongonly}
        )
        options = {'ftol' : 1e-20, 'maxiter': 999}
        result = minimize(fun=RiskParityComputing.riskParityObjective,
                          x0=x0,
                          constraints=constraints,
                          options=options)
        return result.x

In [ ]:
def riskParityOptimization(numpy_matrix):
    weight_Risk_Parity = ModelParameters.zeroWeightMat(time_observed=T2, cols=M)
    return_risk_parity = ModelParameters.zeroFillMat(list_vector=["r_rp"], time_observed=T2)["r_rp"]
    ret_Risk_Parity = ModelParameters.zeroFillMat(list_vector=["retRP"], time_observed=T2)["retRP"]
    print("Running Risk Parity Optimization...")
    for w in range(start_month, end_month):
        weight_Risk_Parity[w - start_month] = PortfolioOptimization.runRiskParity()
        return_risk_parity[w - start_month] = np.dot(numpy_matrix[w, :], weight_Risk_Parity[w - start_month, :])
        ret_Risk_Parity[w - start_month] = sum(return_risk_parity[w - start_month])
    print("DONE!\n")
    return {"w_RP": weight_Risk_Parity, "r_rp": return_risk_parity, "retRP": ret_Risk_Parity}

In [ ]:
riskParityOptimization(numpy_matrix=numpy_matrix)["retRP"]